In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from IPython.display import IFrame
import ipywidgets as widgets

In [5]:
filepath = 'example.html'
IFrame(filepath, width=800, height=700)